In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriasis"
cohort = "GSE123088"

# Input paths
in_trait_dir = "../../input/GEO/Psoriasis"
in_cohort_dir = "../../input/GEO/Psoriasis/GSE123088"

# Output paths
out_data_file = "../../output/preprocess/Psoriasis/GSE123088.csv"
out_gene_data_file = "../../output/preprocess/Psoriasis/gene_data/GSE123088.csv"
out_clinical_data_file = "../../output/preprocess/Psoriasis/clinical_data/GSE123088.csv"
json_path = "../../output/preprocess/Psoriasis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: CD4+ T cells'], 1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female'

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable
import numpy as np
import os
import json

# 1. Gene Expression Data Availability
# From the background information, this appears to be a SuperSeries with multiple datasets
# containing gene expression data from CD4+ T cells, so gene data is likely available
is_gene_available = True

# 2.1 Data Availability
# For trait (Psoriasis), look at row 1 which contains primary diagnosis
trait_row = 1

# For age, look at rows 3 and 4 which contain age information
age_row = 3  # We'll use row 3 as the primary age row

# For gender, look at rows 2 and 3 which contain sex information
gender_row = 2  # Row 2 appears to have more gender information

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait data to binary format (1 for Psoriasis, 0 for others)"""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Check if value matches Psoriasis
    if value.upper() == 'PSORIASIS':
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age data to continuous format"""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)"""
    if pd.isna(value):
        return None
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Check if value matches male or female
    if value.upper() == 'MALE':
        return 1
    elif value.upper() == 'FEMALE':
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Since we don't have direct access to the clinical_data.csv file,
    # we'll use the sample characteristics dictionary from the previous step
    # Create a sample characteristic dictionary based on the provided information
    sample_char_dict = {
        0: ['cell type: CD4+ T cells'], 
        1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 
            'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 
            'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 
            'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 
            'primary diagnosis: TYPE_1_DIABETES', 'primary diagnosis: ACUTE_TONSILLITIS', 
            'primary diagnosis: ULCERATIVE_COLITIS', 'primary diagnosis: Breast cancer', 'primary diagnosis: Control'], 
        2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 
            'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 
            'diagnosis2: OBESITY'], 
        3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 'age: 80', 
            'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 'age: 81', 'age: 70', 
            'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 
            'age: 50', 'age: 24', 'age: 42'], 
        4: [np.nan, 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 'age: 12', 'age: 27']
    }
    
    # Convert sample_char_dict to a DataFrame format that geo_select_clinical_features can use
    clinical_data = pd.DataFrame.from_dict(sample_char_dict, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0, 56.0, 1.0], 1: [0.0, nan, nan], 2: [0.0, 20.0, 0.0], 3: [0.0, 51.0, nan], 4: [0.0, 37.0, nan], 5: [0.0, 61.0, nan], 6: [0.0, 31.0, nan], 7: [0.0, 41.0, nan], 8: [0.0, 80.0, nan], 9: [1.0, 53.0, nan], 10: [0.0, 73.0, nan], 11: [0.0, 60.0, nan], 12: [0.0, 76.0, nan], 13: [0.0, 77.0, nan], 14: [0.0, 74.0, nan], 15: [0.0, 69.0, nan], 16: [nan, 81.0, nan], 17: [nan, 70.0, nan], 18: [nan, 82.0, nan], 19: [nan, 67.0, nan], 20: [nan, 78.0, nan], 21: [nan, 72.0, nan], 22: [nan, 66.0, nan], 23: [nan, 36.0, nan], 24: [nan, 45.0, nan], 25: [nan, 65.0, nan], 26: [nan, 48.0, nan], 27: [nan, 50.0, nan], 28: [nan, 24.0, nan], 29: [nan, 42.0, nan]}
Clinical data saved to ../../output/preprocess/Psoriasis/clinical_data/GSE123088.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')

Gene data dimensions: 24166 genes × 204 samples


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# These appear to be numeric IDs (1, 2, 3, etc.) which are not human gene symbols
# They are likely probe or feature identifiers from the microarray platform
# These would require mapping to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)
print("Gene annotation dataframe:")
print(f"Shape: {gene_annotation.shape}")
print(f"Columns: {gene_annotation.columns.tolist()}")

# 3. Preview the gene annotation dataframe as a Python dictionary
gene_annotation_preview = {col: gene_annotation[col].head(5).tolist() for col in gene_annotation.columns}
print("\nGene annotation preview as dictionary:")
print(gene_annotation_preview)

# 4. Also check platform annotation section for additional context
print("\nSearching for platform annotation section in SOFT file...")
with gzip.open(soft_file, 'rt') as f:
    platform_lines = []
    capture = False
    for i, line in enumerate(f):
        if "^PLATFORM" in line:
            capture = True
            platform_lines.append(line.strip())
        elif capture and line.startswith("!platform_table_begin"):
            platform_lines.append(line.strip())
            for j in range(10):  # Capture the next 10 lines to understand the table structure
                try:
                    platform_line = next(f).strip()
                    platform_lines.append(platform_line)
                except StopIteration:
                    break
            break
    
    print("\n".join(platform_lines))

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe:
Shape: (4740924, 3)
Columns: ['ID', 'ENTREZ_GENE_ID', 'SPOT_ID']

Gene annotation preview as dictionary:
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}

Searching for platform annotation section in SOFT file...
^PLATFORM = GPL25864
!platform_table_begin
ID	ENTREZ_GENE_ID	SPOT_ID
1	1	1
2	2	2
3	3	3
9	9	9
10	10	10
12	12	12
13	13	13
14	14	14
15	15	15


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyzing the structure of the data to understand why mapping isn't working
print("First few rows of gene_data:")
print(gene_data.head())

# 2. Let's check the gene mapping process more carefully
# Create a simplified mapping approach using the Entrez Gene IDs directly
entrez_mapping = gene_annotation[['ID', 'ENTREZ_GENE_ID']].dropna().astype({'ID': 'str'})
entrez_mapping = entrez_mapping.rename(columns={'ENTREZ_GENE_ID': 'Gene'})

print("Entrez mapping sample:")
print(entrez_mapping.head())

# 3. Apply a direct mapping approach - merge the gene expression data with the mapping
gene_data_with_entrez = gene_data.reset_index()
gene_data_with_entrez = pd.merge(gene_data_with_entrez, entrez_mapping, on='ID', how='inner')
gene_data_with_entrez.set_index('Gene', inplace=True)
gene_data_with_entrez.drop('ID', axis=1, inplace=True)

print(f"Gene expression data after direct mapping: {gene_data_with_entrez.shape[0]} genes × {gene_data_with_entrez.shape[1]} samples")
print("First few gene identifiers:")
print(gene_data_with_entrez.index[:10])

# 4. Group by gene ID to handle cases where multiple probes map to the same gene
gene_data = gene_data_with_entrez.groupby(level=0).mean()

print(f"Final gene data after aggregation: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few gene symbols in final data:")
print(gene_data.index[:10])

# 5. Save gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

First few rows of gene_data:
    GSM3494884  GSM3494885  GSM3494886  GSM3494887  GSM3494888  GSM3494889  \
ID                                                                           
1     6.948572    6.783931    6.780049    7.159348    7.311038    8.522366   
2     4.267766    3.692029    3.649207    4.331090    3.903376    4.191000   
3     4.334513    3.981417    3.922257    4.549893    4.140639    4.013236   
9     7.140005    7.411580    6.722240    7.117540    6.874786    7.020385   
10    4.486670    4.615900    3.966261    4.543677    4.150289    4.216883   

    GSM3494890  GSM3494891  GSM3494892  GSM3494893  ...  GSM3495078  \
ID                                                  ...               
1     7.208509    7.339519    7.292977    7.244630  ...    6.510166   
2     3.771017    4.293601    3.963668    4.104845  ...    3.160329   
3     3.996610    4.933703    3.735861    3.992670  ...    3.477307   
9     7.031787    6.980904    7.003364    6.794992  ...    6.126888  

Entrez mapping sample:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10


Gene expression data after direct mapping: 4740720 genes × 204 samples
First few gene identifiers:
Index(['1', '6.948572124', '6.783931247', '6.780049494', '7.159347576',
       '7.31103756', '8.522365778', '7.208509095', '7.339519183',
       '7.292977237'],
      dtype='object', name='Gene')


Final gene data after aggregation: 1584674 genes × 204 samples
First few gene symbols in final data:
Index(['-0.001001579', '-0.010425045', '-0.045320336', '-0.046967808',
       '-0.069891929', '-0.081080626', '-0.093186199', '-0.098238043',
       '-0.103844598', '-0.126973525'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Psoriasis/gene_data/GSE123088.csv
